0 -> Negative; 1 -> Neutral; 2 -> Positive

In [1]:
import pandas as pd

df = pd.read_csv("./data/1000texts.csv", encoding='utf-8-sig')
df.head(3)

,Name,Handle,Timestamp,Verified,Content,Comments,Retweets,Likes,Analytics,Tags,Mentions,Emojis,Profile Image,Tweet Link,Tweet ID
0,Binance,@binance,2024-04-03T00:00:06.000Z,True,The #Binance towel comes everywhere with me......,2.2K,589,2.1K,240K,['#Binance'],[],['\\U0001f373'],https://pbs.twimg.com/profile_images/174428939...,https://twitter.com/binance/status/17753122840...,tweet_id:1775312284052554156
1,Ash Crypto,@Ashcryptoreal,2024-04-04T00:24:48.000Z,True,Drop your $SOL address below and\nmake sure yo...,2.2K,518,1.4K,104K,[],[],['\\U0001f447\\U0001f3fc'],https://pbs.twimg.com/profile_images/169999220...,https://twitter.com/Ashcryptoreal/status/17756...,tweet_id:1775680884982616105
2,Bitcoin,@Bitcoin,2024-04-03T03:19:33.000Z,True,"£52,356.70",156,141,767,161K,[],[],[],https://pbs.twimg.com/profile_images/421692600...,https://twitter.com/Bitcoin/status/17753624737...,tweet_id:1775362473790447725


In [2]:
df = df.dropna()
sentences = df['Content'].tolist()

In [3]:
grouped_list = [sentences[n:n+10] for n in range(0, len(sentences), 10)]

In [6]:
import requests

API_TOKEN = "###"
API_URL = "https://api-inference.huggingface.co/models/cardiffnlp/twitter-roberta-base-sentiment"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output=[]
for i in range(len(grouped_list)):
	output.append(query(grouped_list[i]))
	

In [7]:
output

[[[{'label': 'LABEL_1', 'score': 0.6790464520454407},
   {'label': 'LABEL_2', 'score': 0.2877237796783447},
   {'label': 'LABEL_0', 'score': 0.033229708671569824}],
  [{'label': 'LABEL_1', 'score': 0.7344546318054199},
   {'label': 'LABEL_2', 'score': 0.2018604576587677},
   {'label': 'LABEL_0', 'score': 0.0636848583817482}],
  [{'label': 'LABEL_1', 'score': 0.7219901084899902},
   {'label': 'LABEL_0', 'score': 0.19005246460437775},
   {'label': 'LABEL_2', 'score': 0.08795742690563202}],
  [{'label': 'LABEL_2', 'score': 0.9080163240432739},
   {'label': 'LABEL_1', 'score': 0.08823398500680923},
   {'label': 'LABEL_0', 'score': 0.0037496844306588173}],
  [{'label': 'LABEL_2', 'score': 0.5220183730125427},
   {'label': 'LABEL_1', 'score': 0.3415689766407013},
   {'label': 'LABEL_0', 'score': 0.1364126354455948}],
  [{'label': 'LABEL_1', 'score': 0.5940408110618591},
   {'label': 'LABEL_0', 'score': 0.24663811922073364},
   {'label': 'LABEL_2', 'score': 0.1593211144208908}],
  [{'label': 

In [10]:
highest_labels=[]
for group in output:
    for result in group:
        highest = max(result, key=lambda x: x['score'])
        highest_labels.append(highest['label'].split('_')[1])

In [11]:
df['label'] = highest_labels

In [12]:
import os

df_final = df[['Content', 'label']]

file_path = os.path.join('data', 'labeled_texts_1000.csv')
df_final.to_csv(file_path, index=False, encoding='utf-8-sig')